In [1]:
#Python3 notebook

#from __future__ import absolute_import, division, print_function

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import layers
import shutil
print(tf.keras.__version__)
#import keras

tf.VERSION

W0521 23:49:06.412894 140592504940352 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


2.1.6-tf


'1.12.0'

In [19]:
# import tensorflow.keras.backend as K

feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2" #@param {type:"string"}
def feature_extractor(x):
  feature_extractor_module = hub.Module(feature_extractor_url)
  return feature_extractor_module(x)

# model_prime=tf.keras.models.load_model(
#     "my_model_keras.h5"
# )
# model_prime.summary()

model_prime = tf.contrib.saved_model.load_keras_model("./saved_models/1558364167")
model_prime.summary()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0521 01:15:19.155373 139866154616640 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 6,405
Trainable params: 6,405
Non-trainable params: 0
_________________________________________________________________


In [2]:
class FeatureLayer(tf.layers.Layer):
    def __init__(self, **kwargs):
        self.trainable = False
        super(FeatureLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.feature_vector = hub.Module(
            "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/3",
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )
        super(FeatureLayer, self).build(input_shape)

    def call(self, inputs):
        results = self.feature_vector(inputs=inputs)
        return results

In [19]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/3" #@param {type:"string"}

new_model = tf.contrib.saved_model.load_keras_model("./new/saved/1558446274")
new_model.summary()
# new_model = tf.keras.models.load_model('my_new_model.h5',custom_objects={'FeatureLayer': FeatureLayer()})
# new_model.summary()

ValueError: Unknown layer: FeatureLayer

In [11]:
import tensorflow.keras.backend as K
sess = K.get_session()
init = tf.global_variables_initializer()
sess.run(init)

In [15]:
# import tensorflow.keras.backend as K
# sess = K.get_session()
# # init = tf.global_variables_initializer()

# sess.run(tf.global_variables_initializer())


import numpy as np
import PIL.Image as Image

#print(IMAGE_SIZE)
IMAGE_SIZE=[224, 224]

# result = model.predict(image_batch)
# result.shape
label_names = ['Daisy' ,'Dandelion' ,'Roses', 'Sunflowers' ,'Tulips']

#label_names = np.array([key.title() for key, value in label_names])
print(label_names)
# predicted_class = np.argmax(result[0], axis=-1)
# predicted_class
grace_hopper = "/home/tom/.keras/datasets/flower_photos/sunflowers/6953297_8576bf4ea3.jpg"
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SIZE)
grace_hopper
grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

result = new_model.predict(grace_hopper[np.newaxis, ...])
result.shape
print(result[0])

print(label_names[np.argmax(result[0], axis=-1)])

['Daisy', 'Dandelion', 'Roses', 'Sunflowers', 'Tulips']
[0.2981994  0.4707028  0.1194042  0.07305524 0.03863847]
Dandelion
